In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def binning_custom(value):
    if value >=20 and value <30:
        return 'bin_1'
    elif value >=30  and value <40:
        return 'bin_2'
    elif value >=40  and value <50:
        return 'bin_3'
    elif value >=50  and value <60:
        return 'bin_4'

In [ ]:
traning_df = pd.read_csv('../input/WNS_Train.csv')
traning_df ['age']= traning_df['age'].apply(binning_custom)
traning_df.head()

In [ ]:
test_df = pd.read_csv('../input/WNS_test.csv')
test_df ['age']= test_df['age'].apply(binning_custom)
def create_test_dataframe(train_df):
    train_df['education']=train_df.education.fillna('NO_EDU_DET')
    train_df['previous_year_rating'] = train_df.previous_year_rating.fillna(0)
    department_dummies = pd.get_dummies(train_df['department'])
    train_df = pd.concat([train_df, department_dummies], axis=1)
    education_dummies = pd.get_dummies(train_df['education'])
    train_df = pd.concat([train_df, education_dummies], axis=1)
    gender_dummies = pd.get_dummies(train_df['gender'])
    train_df = pd.concat([train_df, gender_dummies], axis=1)
    recruitment_channel_dummies = pd.get_dummies(train_df['recruitment_channel'])
    train_df = pd.concat([train_df, recruitment_channel_dummies], axis=1)
    
    rating_dummies = pd.get_dummies(train_df['previous_year_rating'],prefix="rating")
    train_df = pd.concat([train_df, rating_dummies], axis=1)
    
    age_dummies = pd.get_dummies(train_df['age'])
    train_df = pd.concat([train_df, age_dummies], axis=1)
    
    train_df = train_df.drop(['department','region','education','gender','recruitment_channel','employee_id','age','previous_year_rating'],axis=1)
    return train_df

test_df = create_test_dataframe(test_df)

In [ ]:
def create_dataframe(train_df):
    train_df['education']=train_df.education.fillna('NO_EDU_DET')
    train_df['previous_year_rating'] = train_df.previous_year_rating.fillna(0)
    department_dummies = pd.get_dummies(train_df['department'])
    train_df = pd.concat([train_df, department_dummies], axis=1)
    education_dummies = pd.get_dummies(train_df['education'])
    train_df = pd.concat([train_df, education_dummies], axis=1)
    gender_dummies = pd.get_dummies(train_df['gender'])
    train_df = pd.concat([train_df, gender_dummies], axis=1)
    recruitment_channel_dummies = pd.get_dummies(train_df['recruitment_channel'])
    train_df = pd.concat([train_df, recruitment_channel_dummies], axis=1)
    
    rating_dummies = pd.get_dummies(train_df['previous_year_rating'],prefix="rating")
    train_df = pd.concat([train_df, rating_dummies], axis=1)
    
    age_dummies = pd.get_dummies(train_df['age'])
    train_df = pd.concat([train_df, age_dummies], axis=1)
    
    train_df = train_df.drop(['department','region','education','gender','recruitment_channel','employee_id','age','previous_year_rating'],axis=1)
    promotoed_df = train_df[train_df.is_promoted == 0]
    not_promoted_df = train_df[train_df.is_promoted == 1]
    subset_df =pd.concat([promotoed_df,not_promoted_df])
    subset_df = subset_df.reset_index(drop=True)
    subset_df = subset_df.reindex(np.random.permutation(subset_df.index))
    return subset_df

In [ ]:
main_df = create_dataframe(traning_df)

main_df.rename(mapper={"Bachelor's":'Bachelor',"Below Secondary":'Below_Secondary',"Master's & above":'master_above',
                       "KPIs_met >80%":"KPI","R&D":"R_N_D","Sales & Marketing":"Sales_Marketing","awards_won?":"awards_won"
                              },axis=1,inplace=True)
selected_features = ['no_of_trainings', 'length_of_service', 'KPI', 'awards_won',
       'avg_training_score', 'Analytics', 'Finance', 'HR',
       'Legal', 'Operations', 'Procurement', 'R_N_D', 'Sales_Marketing',
       'Technology', 'Bachelor', 'Below_Secondary', 'master_above',
       'NO_EDU_DET', 'f', 'm', 'other', 'referred', 'sourcing', 'rating_0.0',
       'rating_1.0', 'rating_2.0', 'rating_3.0', 'rating_4.0', 'rating_5.0',
       'bin_1', 'bin_2', 'bin_3', 'bin_4']
selected_target = ['is_promoted']

In [ ]:
main_df.columns

In [ ]:
from sklearn.model_selection import train_test_split
training_examples, validation_examples, training_targets, validation_targets = train_test_split(main_df[selected_features], main_df[selected_target],
                                                    test_size = 0.2, random_state = 1)

**Repeated Edited Nearest Neighbour**

In [ ]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours

In [ ]:
print(" counts of label '1': {}".format((training_targets==1).sum()))
print(" counts of label '0': {} \n".format((training_targets==0).sum()))

rus = RepeatedEditedNearestNeighbours(size_ngh=5,random_state=1)
X_train_res, y_train_res = rus.fit_sample(training_examples, training_targets)

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

training_examples = pd.DataFrame(X_train_res,columns=training_examples.columns)
training_targets = pd.DataFrame(y_train_res,columns=training_targets.columns)

random_index = np.random.permutation(training_examples.index)
training_examples = training_examples.reindex(random_index)
training_targets = training_targets.reindex(random_index)

In [ ]:
from sklearn.metrics import f1_score
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [ ]:
def run_lgb(X_train, X_test, y_train, y_test, test_df):
    params = {
        "objective" : "binary",
        "n_estimators":10000,
        "reg_alpha" : 0.041545473,
        "reg_lambda":0.1,
        "n_jobs":-1,
        "colsample_bytree":.9497036,
        "min_child_weight":39.3259775,
        "subsample":0.8715623,
        "min_data_in_leaf":300,
        "nthread":4,
        "metric" : "f1",
        "num_leaves" : 600,
        "learning_rate" : 0.01,
        "verbosity" : -1,
        "seed": 120,
        "max_bin":150,
        'max_depth':13,
        'min_gain_to_split':.0222415
    }
    
    lgtrain = lgb.Dataset(X_train, label=y_train)
    lgval = lgb.Dataset(X_test, label=y_test)
    evals_result = {}
    model = lgb.train(params, lgtrain, 10000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=100, 
                      evals_result=evals_result,feval=lgb_f1_score)
    pred_test_y = model.predict(test_df, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [ ]:
import lightgbm as lgb
pred_test, model, evals_result = run_lgb(training_examples,validation_examples,training_targets['is_promoted'],validation_targets['is_promoted'],test_df)
print("LightGBM Training Completed...")

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
x = pca.fit_transform(training_examples)
y = training_targets['is_promoted'].astype('str') 

y[y=='1'] = 'S'
y[y=='0'] = 'L'
x1 , x2 = x[y=='S'],x[(y=='L')]

import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.scatter(list(zip(*x2))[0],list(zip(*x2))[1],color='g')
plt.scatter(list(zip(*x1))[0],list(zip(*x1))[1],color='r',alpha=0.5)
x_cor = list(zip(*x1))[0] + list(zip(*x2))[0]
y_cor = list(zip(*x1))[1] + list(zip(*x2))[1]
plt.axis([min(x_cor),max(x_cor),min(y_cor),max(y_cor)])
plt.show()

**SMOTETomek**

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
training_examples, validation_examples, training_targets, validation_targets = train_test_split(main_df[selected_features], main_df[selected_target],
                                                    test_size = 0.2, random_state = 1)

In [ ]:
print(" counts of label '1': {}".format((training_targets==1).sum()))
print(" counts of label '0': {} \n".format((training_targets==0).sum()))

rus = SMOTETomek(ratio=0.5,k=5,random_state=1)
X_train_res, y_train_res = rus.fit_sample(training_examples, training_targets)

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

training_examples = pd.DataFrame(X_train_res,columns=training_examples.columns)
training_targets = pd.DataFrame(y_train_res,columns=training_targets.columns)

random_index = np.random.permutation(training_examples.index)
training_examples = training_examples.reindex(random_index)
training_targets = training_targets.reindex(random_index)

In [ ]:
pred_test1, model, evals_result = run_lgb(training_examples,validation_examples,training_targets['is_promoted'],validation_targets['is_promoted'],test_df)
print("LightGBM Training Completed...")

In [ ]:
pca=PCA(n_components=2)
x = pca.fit_transform(training_examples)
y = training_targets['is_promoted'].astype('str') 

y[y=='1'] = 'S'
y[y=='0'] = 'L'
x1 , x2 = x[y=='S'],x[(y=='L')]

import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.scatter(list(zip(*x2))[0],list(zip(*x2))[1],color='g')
plt.scatter(list(zip(*x1))[0],list(zip(*x1))[1],color='r',alpha=0.5)
x_cor = list(zip(*x1))[0] + list(zip(*x2))[0]
y_cor = list(zip(*x1))[1] + list(zip(*x2))[1]
plt.axis([min(x_cor),max(x_cor),min(y_cor),max(y_cor)])
plt.show()

**SMOTEENN**

In [ ]:
from imblearn.combine import SMOTEENN

In [ ]:
training_examples, validation_examples, training_targets, validation_targets = train_test_split(main_df[selected_features], main_df[selected_target],
                                                    test_size = 0.2, random_state = 1)

In [ ]:
print(" counts of label '1': {}".format((training_targets==1).sum()))
print(" counts of label '0': {} \n".format((training_targets==0).sum()))

rus = SMOTEENN(ratio=0.5,k=5,random_state=1,size_ngh=5)
X_train_res, y_train_res = rus.fit_sample(training_examples, training_targets)

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

training_examples = pd.DataFrame(X_train_res,columns=training_examples.columns)
training_targets = pd.DataFrame(y_train_res,columns=training_targets.columns)

random_index = np.random.permutation(training_examples.index)
training_examples = training_examples.reindex(random_index)
training_targets = training_targets.reindex(random_index)

In [ ]:
pred_test, model, evals_result = run_lgb(training_examples,validation_examples,training_targets['is_promoted'],validation_targets['is_promoted'],test_df)
print("LightGBM Training Completed...")

In [ ]:
pca=PCA(n_components=2)
x = pca.fit_transform(training_examples)
y = training_targets['is_promoted'].astype('str') 

y[y=='1'] = 'S'
y[y=='0'] = 'L'
x1 , x2 = x[y=='S'],x[(y=='L')]

import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.scatter(list(zip(*x2))[0],list(zip(*x2))[1],color='g')
plt.scatter(list(zip(*x1))[0],list(zip(*x1))[1],color='r',alpha=0.5)
x_cor = list(zip(*x1))[0] + list(zip(*x2))[0]
y_cor = list(zip(*x1))[1] + list(zip(*x2))[1]
plt.axis([min(x_cor),max(x_cor),min(y_cor),max(y_cor)])
plt.show()

In [ ]:
pred_test1

In [ ]:
final_pred = (pred_test + pred_test1)/2

In [ ]:
employee_df = pd.read_csv('../input/WNS_test.csv')

In [ ]:
validation_df = pd.DataFrame(final_pred,columns=["is_promoted"])
validation_df["is_promoted"]  = validation_df["is_promoted"].apply(lambda x: 1 if x > 0.5 else 0)
validation_df['employee_id'] = employee_df['employee_id']
validation_df= validation_df[['employee_id','is_promoted']]

In [ ]:
validation_df.to_csv('sampling_avg.csv',index=False)